I   ПОДГОТОВКА ТЕКСТА  И   ОБУЧЕНИЕ МОДЕЛИ

???   В обучении нет явной проверки рифмы между 2 и 4 строкой, только по корпусу.
??? Нет разделения на train/test, но для маленьких датасетов это допустимо. 

In [ ]:
# ===== Шаг 1. Установка пакетов (если нужно) =====
#   %pip install transformers datasets accelerate   ((fsspec-2025.7.0))
 


# ===============================================================
""" 
    Подготовка database, обучение и генерации порошков.  Фильтрация.  
"""

# ===== Шаг 3. Датасет =====
from datasets import Dataset

dataset = Dataset.from_dict({"text": unique_poems})
# print(dataset)   ??? этот принт не нужен? 

# ===== Шаг 4. Токенизация =====
from transformers import AutoTokenizer

model_name = "sberbank-ai/rugpt3medium_based_on_gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize(batch):
    return tokenizer(batch["text"], truncation=True, padding="max_length", max_length=64)
# max_length=64  ??? 128? 

tokenized = dataset.map(tokenize, batched=True, remove_columns=["text"])
# print(tokenized)   ??? этот принт не нужен?

# ===== Шаг 5. Fine-tuning =====
from transformers import AutoModelForCausalLM, TrainingArguments, Trainer
import torch

model = AutoModelForCausalLM.from_pretrained(model_name)

fp16 = torch.cuda.is_available() # если есть GPU с поддержкой — можно True

training_args = TrainingArguments(
    output_dir="poroshki_model",
    overwrite_output_dir=True,
    num_train_epochs=3,               # ???  можно увеличить
    per_device_train_batch_size=1,    # ноут не потянет больше
    save_steps=500,
    save_total_limit=2,
    logging_dir="./logs",
    logging_steps=50,
    learning_rate=5e-5,
    fp16=fp16                        # если есть GPU с поддержкой — можно True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized,
)

trainer.train()

# ===== Шаг 6. Сохранение =====
trainer.save_model("poroshki_model")
tokenizer.save_pretrained("poroshki_model")

print("Модель сохранена в папке: poroshki_model/")

II  ГЕНЕРАЦИЯ

In [ ]:
import re
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from ruaccent import RUAccent

# ======== Модель ======== 
model_name = "sberbank-ai/rugpt3medium_based_on_gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)


# ======== Твой фильтр ========

# Настройка RUAccent
accentizer = RUAccent()
accentizer.load(omograph_model_size='turbo3.1',
                use_dictionary=True,
                tiny_mode=False)

VOWELS = "аеёиоуыэюяАЕЁИОУЫЭЮЯ"
SCHEME = [9, 8, 9, 2]  # схема по слогам


def accentize_text(lines: list[str]) -> list[str]:
    return [accentizer.process_all(line) for line in lines]


def plus_to_upper(lines: list[str]) -> list[str]:
    result = []
    for line in lines:
        chars = list(line)
        out = []
        skip = False
        for i, ch in enumerate(chars):
            if skip:
                skip = False
                continue
            if ch == "+" and i + 1 < len(chars) and chars[i + 1] in VOWELS:
                out.append(chars[i + 1].upper())
                skip = True
            else:
                out.append(ch)
        result.append("".join(out))
    return result


def accent_one_and_yo(lines: list[str]) -> list[str]:
    result = []
    for line in lines:
        chars = list(line)
        vowel_positions = [i for i, ch in enumerate(
            chars) if ch.lower() in VOWELS]
        vowel_order = {pos: idx for idx,
                       pos in enumerate(vowel_positions, start=1)}

        new_words = []
        offset = 0
        for word in line.split():
            word_chars = list(word)
            local_vowels = [i for i, ch in enumerate(
                word_chars) if ch.lower() in VOWELS]
            if len(local_vowels) == 1:  # односложное слово
                i = local_vowels[0]
                ch = word_chars[i]
                vpos_in_line = offset + i
                order = vowel_order.get(vpos_in_line)
                if order is not None:
                    if order % 2 == 0:  # чётный
                        word_chars[i] = ch.upper()
                    else:  # нечётный
                        word_chars[i] = ch.lower()
            elif len(local_vowels) > 1:  # несколько гласных
                for i, ch in enumerate(word_chars):
                    if ch == "ё":
                        word_chars[i] = "Ё"
            new_words.append(''.join(word_chars))
            offset += len(word) + 1
        result.append(' '.join(new_words))
    return result


def count_syllables(line: str) -> int:
    return sum(1 for ch in line if ch in VOWELS)


def line_stresses(line: str) -> list[int]:
    stresses = []
    syllable = 0
    for ch in line:
        if ch in VOWELS:
            syllable += 1
            if ch.isupper():
                stresses.append(syllable)
    return stresses


def analyze_poem_and_accents(accented_lines: list[str]):
    scheme_syllables = [count_syllables(line) for line in accented_lines]
    scheme_accents = [line_stresses(line) for line in accented_lines]
    return scheme_syllables, scheme_accents


def check_form(accented_lines: list[str], scheme_ref: list[int]):
    SCHEME_SYLLABLES, SCHEME_ACCENTS = analyze_poem_and_accents(
        accented_lines)

    mismatches = []
    for i, syll_count in enumerate(SCHEME_SYLLABLES):
        expected = scheme_ref[i] if i < len(scheme_ref) else None
        if expected is None:
            continue
        if syll_count != expected:
            mismatches.append({
                "line_no": i + 1,
                "line": accented_lines[i],
                "actual": syll_count,
                "expected": expected
            })

    iamb_failures = []
    for i, accents in enumerate(SCHEME_ACCENTS):
        odd_accents = [a for a in accents if a % 2 == 1]
        if odd_accents:
            iamb_failures.append({
                "line_no": i + 1,
                "line": accented_lines[i],
                "odd_accents": odd_accents,
                "all_accents": accents
            })

    return {
        "SCHEME_SYLLABLES": SCHEME_SYLLABLES,
        "SCHEME_ACCENTS": SCHEME_ACCENTS,
        "mismatches": mismatches,
        "iamb_failures": iamb_failures
    }

# ======== Проверка порошка ========
def validate_poroshok(poem: str) -> bool:
    lines = [l.strip() for l in poem.split("\n") if l.strip()]
    if len(lines) != 4:
        return False
    try:
        accented = accentize_text(lines)
        accented_big = plus_to_upper(accented)
        accented_all = accent_one_and_yo(accented_big)
        result = check_form(accented_all, SCHEME)
        if result["mismatches"] or result["iamb_failures"]:
            return False
        return True
    except Exception as e:
        print("Ошибка акцентуации:", e)
        return False

# ======== Генерация ========
def generate_poroshki(prompt=""):
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)
    outputs = model.generate(
        input_ids,
        max_length=64,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        temperature=0.9,
        num_return_sequences=10, # количество вариантов ???
        pad_token_id=tokenizer.eos_token_id
    )
    texts = [tokenizer.decode(o, skip_special_tokens=True) for o in outputs]
    poroshki = []
    for t in texts:
        blocks = re.split(r"\n\s*\n", t)
        for b in blocks:
            if validate_poroshok(b):
                poroshki.append(b)
    return poroshki


# ======== Запуск ========
good = generate_poroshki("")

print("Подходящие порошки:")
for p in good:
    print("—" * 40)
    print(p)